# Data Wrangling

### 1 Table of Contents<a id='Contents'></a>
* [Data Wrangling](#Data_Wrangling)
  * [1 Contents](#Contents)
  * [2 Introduction](#2_Introduction)
      * [2.1 Problem](#2.1_Problem)
      * [2.2 Data Sources](#2.2_Data_Sources)
  * [3 Imports](#3_Imports)
  * [4 Load Data](#4_Load_Data)
      * [4.1 Highway Collisions](#4.1_Highway_Collisions)
      * [4.2 Non-Highway Collisions](#4.2_Non-Highway_Collisions)
      * [4.3 Weather](#4.3_Weather)
  * [5 Traffic Collisions](#5_Traffic_Collisions)
      * [5.1 Highway Collisions](#5.1_Highway_Collisions)
      * [5.2 Non-Highway Collisions](#5.2_Non-Highway_Collisions)
  * [6 Weather](#6_Weather)
  * [7 Merge](#7_Merge)
      * [7.1 Combining Collision Datasets](#7.1_Combining_Collision_Datasets)
      * [7.2 Merging Collision and Weather](#7.2_Merging_Collision_and_Weather)
  * [8 First Look](#8_First_Look)
  * [9 Google Maps API](#9_Google_Maps_API)
  * [10 OpenWeather API](#10_OpenWeather_API)
  * [11 Save Data](#11_Save_Data)
  * [12 Conclusion](#12_Conclusion)

### 2 Introduction<a id='2_Introduction'>

#### 2.1 Problem<a id='2.1_Problem'>

Millions of people are on the road everyday. In fact, there are more than seven million cars registered in LA county alone. With that many cars on the road, and pedestrians and cyclists, it can be quite dangerous to commute to everyday activities like work and grocery shopping, However, some routes are more dangerous than others. 

I will create a model to return the likelihood of an accident occurring on the different routes returned by the Google Maps API when prompted by a user, such that  the user can accurately choose the safest journey to take more than 75 percent of the time. For this project, I will be solely focusing on the city of Los Angeles.


#### 2.2 Data Sources<a id='2.2_Data_Sources'>

The data that will be used is listed below:
- SWITRS dataset on traffic collisions in California from 2001 to 2022. Sqlite files include info on collisions, parties involved, injuries, and a few weather factors. This is the dataset that will be used for modeling. The set contains over 300,000 collisions in the city of Los Angeles from 2007 to 2021.
- LAPD data on non-highway traffic collisions from 2010 to present.
- OpenWeather API provides weather data used for training the model, as this set has more useful weather factors, mainly visibility, than the California traffic collision dataset.
- Google Maps API to provide routes on demand	to users, who will then be given the likelihood that each route will result in an accident. API 

### 3 Imports<a id='3_Imports'>

In [1]:
import warnings
warnings.simplefilter('ignore')
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from library.sb_utils import save_file
import requests
import gmaps
import googlemaps
import os
import csv
%matplotlib inline

### 4 Load Data<a id='4_Load_Data'>

In [2]:
LA_highway_collisions = pd.read_csv('../Raw Data/Traffic.csv')
LA_nonhighway_collisions = pd.read_csv('../Raw Data/Traffic_Collisions_LAPD.csv')
LA_weather = pd.read_csv('../Raw Data/Los_Angeles_Weather.csv')
openWeather_api_key = pd.read_json('../credentials.json', typ='series')['openWeather_api_key']
maps_api_key = pd.read_json('../credentials.json', typ='series')['maps_api_key']

#### 4.1 Highway Collisions<a id='4.1_Highway_Collisions'>

In [3]:
LA_highway_collisions.head()

,id,road,intersection,side_of_highway,severity,type,pedestrian,bicycle,motorcycle,truck,same_day_crashes,same_road_crashes,latitude,longitude,datetime
0,81488182,CENTINELA AV,0.0,NaN,pain,broadside,0,0,0,0,3,229,33.97074,-118.34786,2021-06-01 13:58:00
1,81488183,MANCHESTER BLVD,1.0,NaN,pain,broadside,0,0,0,0,3,185,33.95995,-118.32868,2021-06-01 16:15:00
2,91488003,I-710 NORTHBOUND(LONG BEACH FREEWAY),0.0,NaN,property damage only,rear end,0,0,0,0,3,1,33.92770,-118.17820,2021-06-01 16:20:00
3,81487205,LA BREA AV,1.0,NaN,property damage only,hit object,0,0,0,0,3,424,33.96550,-118.35330,2021-05-30 06:45:00
4,91487342,I-605 (S/B),0.0,NaN,severe injury,hit object,0,0,0,0,3,131,33.97228,-118.08097,2021-05-30 22:35:00


The collisions dataset contains information on each wreck, including: severity, road, and time. The dataset was queried from the SWITRS database using SQLiteStudio. 

In [4]:
LA_highway_collisions.shape

(319906, 15)

#### 4.2 Non-Highway Collisions<a id='4.2_Non-Highway_Collisions'>

In [5]:
LA_nonhighway_collisions.head()

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Code,Premise Description,Address,Cross Street,Location
0,190319651,08/24/2019,08/24/2019,450,3,Southwest,356,997,TRAFFIC COLLISION,3036 3004 3026 3101 4003,22.0,M,H,101.0,STREET,JEFFERSON BL,NORMANDIE AV,"(34.0255, -118.3002)"
1,190319680,08/30/2019,08/30/2019,2320,3,Southwest,355,997,TRAFFIC COLLISION,3037 3006 3028 3030 3039 3101 4003,30.0,F,H,101.0,STREET,JEFFERSON BL,W WESTERN,"(34.0256, -118.3089)"
2,190413769,08/25/2019,08/25/2019,545,4,Hollenbeck,422,997,TRAFFIC COLLISION,3101 3401 3701 3006 3030,NaN,M,X,101.0,STREET,N BROADWAY,W EASTLAKE AV,"(34.0738, -118.2078)"
3,190127578,11/20/2019,11/20/2019,350,1,Central,128,997,TRAFFIC COLLISION,0605 3101 3401 3701 3011 3034,21.0,M,H,101.0,STREET,1ST,CENTRAL,"(34.0492, -118.2391)"
4,190319695,08/30/2019,08/30/2019,2100,3,Southwest,374,997,TRAFFIC COLLISION,0605 4025 3037 3004 3025 3101,49.0,M,B,101.0,STREET,MARTIN LUTHER KING JR,ARLINGTON AV,"(34.0108, -118.3182)"


In [6]:
LA_nonhighway_collisions.shape

(587194, 18)

#### 4.3 Weather<a id='4.3_Weather'>

In [7]:
LA_weather.head()

,dt,dt_iso,timezone,city_name,lat,lon,temp,visibility,dew_point,feels_like,...,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,1167609600,2007-01-01 00:00:00 +0000 UTC,-28800,Los Angeles,34.052234,-118.243685,15.88,10000.0,8.38,15.12,...,NaN,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01d
1,1167613200,2007-01-01 01:00:00 +0000 UTC,-28800,Los Angeles,34.052234,-118.243685,14.79,10000.0,8.51,14.05,...,NaN,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01n
2,1167616800,2007-01-01 02:00:00 +0000 UTC,-28800,Los Angeles,34.052234,-118.243685,14.01,10000.0,9.45,13.40,...,NaN,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01n
3,1167620400,2007-01-01 03:00:00 +0000 UTC,-28800,Los Angeles,34.052234,-118.243685,12.08,9656.0,6.99,11.20,...,NaN,NaN,NaN,NaN,NaN,100,721,Haze,haze,50n
4,1167624000,2007-01-01 04:00:00 +0000 UTC,-28800,Los Angeles,34.052234,-118.243685,11.55,8047.0,7.66,10.77,...,NaN,NaN,NaN,NaN,NaN,75,721,Haze,haze,50n


The weather dataset contains information on each wreck, including: temp, visibility, and wind. This dataset will be joined to the collisions dataset on the time column later in this notebook. But first, cleaning the datasets.

In [8]:
LA_weather.shape

(132731, 28)

### 5 Traffic Collisions<a id='5_Traffic_Collisions'>

#### 5.1 Highway Collisions<a id='5.1_Highway_Collisions'>

In [9]:
LA_highway_collisions.dtypes

id                     int64
road                  object
intersection         float64
side_of_highway       object
severity              object
type                  object
pedestrian             int64
bicycle                int64
motorcycle             int64
truck                  int64
same_day_crashes       int64
same_road_crashes      int64
latitude             float64
longitude            float64
datetime              object
dtype: object

The object dtypes are string objects. These will be turned into dummy columns in a future notebook. For now, focusing on missing values.

In [10]:
LA_highway_collisions.isna().sum()

id                        0
road                      0
intersection            266
side_of_highway      216113
severity                  0
type                    805
pedestrian                0
bicycle                   0
motorcycle                0
truck                     0
same_day_crashes          0
same_road_crashes         0
latitude                  0
longitude                 0
datetime               1817
dtype: int64

There are quite a lot of missing values in a number of columns. Datetime is important, since that is the column that  will be used to join the two datasets on. Imputing values based on nearby values is an option, however it is only about 0.5% of the rows that are missing values in that column, so they will be dropped instead to avoid incorrect imputation. 

In [11]:
LA_highway_collisions.dropna(subset = ['datetime'], inplace = True)

In [12]:
print('side_of_highway:', LA_highway_collisions['side_of_highway'].unique())
print('type:', LA_highway_collisions['type'].unique())
print('intersection:', LA_highway_collisions['intersection'].unique())

side_of_highway: [nan 'eastbound' 'southbound' 'northbound' 'westbound']
type: ['broadside' 'rear end' 'hit object' 'sideswipe' 'head-on' 'other'
 'pedestrian' 'overturned' nan]
intersection: [ 0.  1. nan]


The above columns are a mixture of string objects and a float. The string objects will be imputed with "Not Available" and the float will be imputed with zero. It is assumed that missing values for intersection are instances that did not involve an intersection

In [13]:
LA_highway_collisions.fillna({'side_of_highway':'Not Available', 'type':'Not Available',
                              'intersection':0}, inplace = True)

In [14]:
LA_highway_collisions.isna().sum()

id                   0
road                 0
intersection         0
side_of_highway      0
severity             0
type                 0
pedestrian           0
bicycle              0
motorcycle           0
truck                0
same_day_crashes     0
same_road_crashes    0
latitude             0
longitude            0
datetime             0
dtype: int64

That's all of the null values handled in this dataset. The id column is no longer needed so that can be dropped.

In [15]:
LA_highway_collisions.drop(columns = ['id'], inplace = True)

In [16]:
LA_highway_collisions.nunique()

road                  26566
intersection              2
side_of_highway           5
severity                  5
type                      9
pedestrian                2
bicycle                   2
motorcycle                2
truck                     2
same_day_crashes        145
same_road_crashes       319
latitude              24841
longitude             43383
datetime             283209
dtype: int64

Now, the non-highway collisions

#### 5.2 Non-Highway Collisions<a id='5.2_Non-Highway_Collisions'>

In [17]:
LA_nonhighway_collisions.dtypes

DR Number                   int64
Date Reported              object
Date Occurred              object
Time Occurred               int64
Area ID                     int64
Area Name                  object
Reporting District          int64
Crime Code                  int64
Crime Code Description     object
MO Codes                   object
Victim Age                float64
Victim Sex                 object
Victim Descent             object
Premise Code              float64
Premise Description        object
Address                    object
Cross Street               object
Location                   object
dtype: object

There are many unneeded columns in this dataset. Mostly, columns pertaining to the victim's age or sex or pertaining to the area, other than Location, are not needed.

In [18]:
LA_nonhighway_collisions.drop(columns = ['DR Number', 'Date Reported', 'Area ID', 'Area Name', 'Reporting District', 
                                         'Crime Code', 'Crime Code Description', 'MO Codes', 'Victim Sex', 
                                         'Victim Age', 'Victim Descent', 'Premise Code', 'Premise Description', 
                                         'Cross Street'], inplace = True)

In [19]:
LA_nonhighway_collisions.head()

,Date Occurred,Time Occurred,Address,Location
0,08/24/2019,450,JEFFERSON BL,"(34.0255, -118.3002)"
1,08/30/2019,2320,JEFFERSON BL,"(34.0256, -118.3089)"
2,08/25/2019,545,N BROADWAY,"(34.0738, -118.2078)"
3,11/20/2019,350,1ST,"(34.0492, -118.2391)"
4,08/30/2019,2100,MARTIN LUTHER KING JR,"(34.0108, -118.3182)"


In [20]:
print(type(LA_nonhighway_collisions['Location'][0]))

<class 'str'>


The Location column has the latitude and logitude as a string. This string can be broken up by using ', ' as the seperator. The type will also be changed to float64 to match the highway collisions dataset

In [21]:
LA_nonhighway_collisions['latitude'] = LA_nonhighway_collisions['Location'].agg(
    lambda x: x.split(', ')[0].replace('(', '')).astype('float64')
LA_nonhighway_collisions['longitude'] = LA_nonhighway_collisions['Location'].agg(
    lambda x: x.split(', ')[1].replace(')', '')).astype('float64')
LA_nonhighway_collisions.drop(columns = ['Location'], inplace = True)

In [22]:
LA_nonhighway_collisions.head()

,Date Occurred,Time Occurred,Address,latitude,longitude
0,08/24/2019,450,JEFFERSON BL,34.0255,-118.3002
1,08/30/2019,2320,JEFFERSON BL,34.0256,-118.3089
2,08/25/2019,545,N BROADWAY,34.0738,-118.2078
3,11/20/2019,350,1ST,34.0492,-118.2391
4,08/30/2019,2100,MARTIN LUTHER KING JR,34.0108,-118.3182


In [23]:
LA_nonhighway_collisions.dtypes

Date Occurred     object
Time Occurred      int64
Address           object
latitude         float64
longitude        float64
dtype: object

Are there any null values?

In [24]:
LA_nonhighway_collisions.isna().sum()

Date Occurred    0
Time Occurred    0
Address          0
latitude         0
longitude        0
dtype: int64

Columns are added to match the highway collisions dataset. The columns are loaded with either 0 or 'Not Available' depending on column type.

In [25]:
LA_nonhighway_collisions['intersection'] = 0
LA_nonhighway_collisions['side_of_highway'] = 'Not Available'
LA_nonhighway_collisions['severity'] = 'Not Available'
LA_nonhighway_collisions['type'] = 'Not Available'
LA_nonhighway_collisions['pedestrian'] = 0
LA_nonhighway_collisions['bicycle'] = 0
LA_nonhighway_collisions['motorcycle'] = 0
LA_nonhighway_collisions['truck'] = 0
LA_nonhighway_collisions['same_day_crashes'] = 0
LA_nonhighway_collisions['same_road_crashes'] = 0

### 6 Weather<a id='6_Weather'>

In [26]:
LA_weather.dtypes

dt                       int64
dt_iso                  object
timezone                 int64
city_name               object
lat                    float64
lon                    float64
temp                   float64
visibility             float64
dew_point              float64
feels_like             float64
temp_min               float64
temp_max               float64
pressure                 int64
sea_level              float64
grnd_level             float64
humidity                 int64
wind_speed             float64
wind_deg                 int64
wind_gust              float64
rain_1h                float64
rain_3h                float64
snow_1h                float64
snow_3h                float64
clouds_all               int64
weather_id               int64
weather_main            object
weather_description     object
weather_icon            object
dtype: object

Again, this dataset, like the previous, is a mixture of numerics and strings.

In [27]:
LA_weather.isna().sum()

dt                          0
dt_iso                      0
timezone                    0
city_name                   0
lat                         0
lon                         0
temp                        0
visibility               1133
dew_point                   0
feels_like                  0
temp_min                    0
temp_max                    0
pressure                    0
sea_level              132731
grnd_level             132731
humidity                    0
wind_speed                  0
wind_deg                    0
wind_gust              112518
rain_1h                126144
rain_3h                132314
snow_1h                132731
snow_3h                132731
clouds_all                  0
weather_id                  0
weather_main                0
weather_description         0
weather_icon                0
dtype: int64

In [28]:
LA_weather.drop(columns = ['dt_iso', 'timezone', 'city_name', 'lat', 'lon', 'sea_level', 'grnd_level', 'feels_like', \
                           'weather_id', 'weather_main', 'weather_description', 'weather_icon'], inplace = True)

The columns dropped above are not pertinent to our this case or for the joining of the two datasets. 

In [29]:
LA_weather['wind_gust'].value_counts()

0.00     8578
0.45     1257
1.34      853
1.79      823
2.24      755
         ... 
12.34       1
14.39       1
17.00       1
16.00       1
12.07       1
Name: wind_gust, Length: 84, dtype: int64

In [30]:
LA_weather['rain_1h'].value_counts()

0.30    699
0.50    413
0.20    300
0.25    242
0.11    197
       ... 
2.44      1
3.32      1
2.31      1
1.79      1
2.95      1
Name: rain_1h, Length: 391, dtype: int64

In [31]:
LA_weather['rain_3h'].value_counts()

0.30     87
0.50     41
0.80     32
1.30     18
1.00     18
         ..
18.80     1
8.60      1
16.80     1
28.40     1
8.12      1
Name: rain_3h, Length: 64, dtype: int64

Looking at the value counts above, it is reasonable to impute the value of zero for null values for those columns.

In [32]:
LA_weather[LA_weather['rain_1h'] == 0].shape

(0, 16)

Indeed, there are no zero values in the rain_1h column, giving further confirmation that the missing values are meant to be zero.

The columns snow_1h and snow_3h are missing all of the values. This isn't that surprising; it is L.A. So, these columns will be dropped, as well.

In [33]:
LA_weather.drop(columns = ['snow_1h', 'snow_3h'], inplace = True)

The missing values in the visibility column will be dropped. Like the datetime column in the collisions dataset, the missing values only make up about 1% of the total number of rows, and imputing could lead to erroneous results

In [34]:
LA_weather.dropna(subset = ['visibility'], inplace = True)
LA_weather.fillna(value = 0, inplace = True)

In [35]:
LA_weather.isna().sum()

dt            0
temp          0
visibility    0
dew_point     0
temp_min      0
temp_max      0
pressure      0
humidity      0
wind_speed    0
wind_deg      0
wind_gust     0
rain_1h       0
rain_3h       0
clouds_all    0
dtype: int64

Again, this dataset is now cleaning of any missing values

In [36]:
LA_weather.nunique()

dt            130363
temp            3479
visibility        19
dew_point       3836
temp_min        3170
temp_max        3644
pressure          49
humidity          98
wind_speed       228
wind_deg         361
wind_gust         84
rain_1h          389
rain_3h           65
clouds_all        98
dtype: int64

### 7 Merge<a id='7_Merge'>

#### 7.1 Combining Collision Datasets<a id='7.1_Combining_Collision_Datasets'>

To start, the 'Date Occurred' and 'Time Occurred' columns of the non-highway collisions dataset need to be combined into a single column. The Time Occured column is an int representation of a 24-hour clock. This column needs to be changed into a workable format 

In [37]:
LA_nonhighway_collisions['Time Occurred'] = LA_nonhighway_collisions['Time Occurred'].astype('str').agg(
    lambda x: x[:2] + ':' + x[2:] + ':00' if len(x) > 3 else x[:1] + ':' + x[1:] + ':00' if
    len(x) > 2 else '00' + ':' + x + ':00')
LA_nonhighway_collisions['Date Occurred'] = LA_nonhighway_collisions['Date Occurred'].agg(
    lambda x: x.replace('/','-'))

In [38]:
LA_nonhighway_collisions['Time Occurred'].head()

0     4:50:00
1    23:20:00
2     5:45:00
3     3:50:00
4    21:00:00
Name: Time Occurred, dtype: object

In [39]:
LA_nonhighway_collisions['datetime'] = LA_nonhighway_collisions['Date Occurred'] + ' ' + LA_nonhighway_collisions['Time Occurred']

In [40]:
LA_nonhighway_collisions.head()

,Date Occurred,Time Occurred,Address,latitude,longitude,intersection,side_of_highway,severity,type,pedestrian,bicycle,motorcycle,truck,same_day_crashes,same_road_crashes,datetime
0,08-24-2019,4:50:00,JEFFERSON BL,34.0255,-118.3002,0,Not Available,Not Available,Not Available,0,0,0,0,0,0,08-24-2019 4:50:00
1,08-30-2019,23:20:00,JEFFERSON BL,34.0256,-118.3089,0,Not Available,Not Available,Not Available,0,0,0,0,0,0,08-30-2019 23:20:00
2,08-25-2019,5:45:00,N BROADWAY,34.0738,-118.2078,0,Not Available,Not Available,Not Available,0,0,0,0,0,0,08-25-2019 5:45:00
3,11-20-2019,3:50:00,1ST,34.0492,-118.2391,0,Not Available,Not Available,Not Available,0,0,0,0,0,0,11-20-2019 3:50:00
4,08-30-2019,21:00:00,MARTIN LUTHER KING JR,34.0108,-118.3182,0,Not Available,Not Available,Not Available,0,0,0,0,0,0,08-30-2019 21:00:00


In [41]:
LA_nonhighway_collisions.drop(columns = ['Date Occurred', 'Time Occurred'], inplace = True)
LA_nonhighway_collisions.rename(columns = {'Address':'road'}, inplace = True)

In [42]:
LA_collisions = LA_highway_collisions.append(LA_nonhighway_collisions, ignore_index = True)

In [43]:
LA_collisions.head()

,road,intersection,side_of_highway,severity,type,pedestrian,bicycle,motorcycle,truck,same_day_crashes,same_road_crashes,latitude,longitude,datetime
0,CENTINELA AV,0.0,Not Available,pain,broadside,0,0,0,0,3,229,33.97074,-118.34786,2021-06-01 13:58:00
1,MANCHESTER BLVD,1.0,Not Available,pain,broadside,0,0,0,0,3,185,33.95995,-118.32868,2021-06-01 16:15:00
2,I-710 NORTHBOUND(LONG BEACH FREEWAY),0.0,Not Available,property damage only,rear end,0,0,0,0,3,1,33.92770,-118.17820,2021-06-01 16:20:00
3,LA BREA AV,1.0,Not Available,property damage only,hit object,0,0,0,0,3,424,33.96550,-118.35330,2021-05-30 06:45:00
4,I-605 (S/B),0.0,Not Available,severe injury,hit object,0,0,0,0,3,131,33.97228,-118.08097,2021-05-30 22:35:00


In [44]:
LA_collisions.shape

(905283, 14)

Great! The two collision datasets are now combined into one dataframe. Now that dataframe can be merged with the weather data.

#### 7.2 Merging Collision and Weather<a id='7.2_Merging_Collision_and_Weather'>

The datasets will be merged on the datetime column. Before this can be done, the weather dataset needs a datetime column. This can be extracted form the dt column which is the time given as a UNIX timestamp.

In [45]:
LA_weather['datetime'] = pd.to_datetime(LA_weather['dt'], unit='s')

In [46]:
LA_weather['datetime'].head(10)

0   2007-01-01 00:00:00
1   2007-01-01 01:00:00
2   2007-01-01 02:00:00
3   2007-01-01 03:00:00
4   2007-01-01 04:00:00
5   2007-01-01 05:00:00
6   2007-01-01 06:00:00
7   2007-01-01 07:00:00
8   2007-01-01 08:00:00
9   2007-01-01 09:00:00
Name: datetime, dtype: datetime64[ns]

However, the datetime column is now in GMT time. pd.Timedelta allows for subtracting the necessary amount of hours, 8, to get the column into L.A. time.

In [47]:
LA_weather.loc[:,'datetime'] += pd.Timedelta(value = -8, unit = 'h')

In [48]:
LA_weather['datetime'].head(10)

0   2006-12-31 16:00:00
1   2006-12-31 17:00:00
2   2006-12-31 18:00:00
3   2006-12-31 19:00:00
4   2006-12-31 20:00:00
5   2006-12-31 21:00:00
6   2006-12-31 22:00:00
7   2006-12-31 23:00:00
8   2007-01-01 00:00:00
9   2007-01-01 01:00:00
Name: datetime, dtype: datetime64[ns]

Since, the weather dataset's datetime column is a datetime object, so should be the collisions dataset's datetime column.

In [49]:
LA_collisions['datetime'].head()

0    2021-06-01 13:58:00
1    2021-06-01 16:15:00
2    2021-06-01 16:20:00
3    2021-05-30 06:45:00
4    2021-05-30 22:35:00
Name: datetime, dtype: object

In [50]:
LA_collisions['datetime'] = pd.to_datetime(LA_collisions['datetime'])

In [51]:
LA_collisions['datetime'].head()

0   2021-06-01 13:58:00
1   2021-06-01 16:15:00
2   2021-06-01 16:20:00
3   2021-05-30 06:45:00
4   2021-05-30 22:35:00
Name: datetime, dtype: datetime64[ns]

The weather dataset's datetime column is in intervals of one hour, while the collisions dataset's datetime values can be any time. Before the merge, the collisions dataset's datetime column will be rounded to the nearest hour. Also, duplicates will be dropped from the weather dataset to avoid any unwanted rows

In [52]:
LA_collisions['datetime'] = LA_collisions['datetime'].dt.round(freq = 'H')
LA_weather.drop_duplicates(subset = ['datetime'], keep = False, inplace = True)

In [53]:
LA_data = LA_collisions.merge(LA_weather, how = 'inner', on = 'datetime')

In [54]:
LA_data.head()

,road,intersection,side_of_highway,severity,type,pedestrian,bicycle,motorcycle,truck,same_day_crashes,...,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,wind_gust,rain_1h,rain_3h,clouds_all
0,CENTINELA AV,0.0,Not Available,pain,broadside,0,0,0,0,3,...,20.21,32.14,1014,54,1.34,293,0.45,0.0,0.0,0
1,CRENSHAW BL,0.0,Not Available,Not Available,Not Available,0,0,0,0,0,...,20.21,32.14,1014,54,1.34,293,0.45,0.0,0.0,0
2,MANCHESTER BLVD,1.0,Not Available,pain,broadside,0,0,0,0,3,...,19.10,31.11,1013,58,2.57,0,0.00,0.0,0.0,0
3,I-710 NORTHBOUND(LONG BEACH FREEWAY),0.0,Not Available,property damage only,rear end,0,0,0,0,3,...,19.10,31.11,1013,58,2.57,0,0.00,0.0,0.0,0
4,JEFFERSON,0.0,Not Available,Not Available,Not Available,0,0,0,0,0,...,19.10,31.11,1013,58,2.57,0,0.00,0.0,0.0,0


In [55]:
LA_data.shape

(870520, 28)

After the merge, the dataset contains only a slightly fewer number of rows than the collisions dataset. This means that only a few accidents didn't have corresponding weather data, which is great.

### 8 First Look<a id='8_First_Look'>

Which 5 roads see the most accidents?

In [56]:
LA_data['road'].value_counts().head(5)

RT 10     17773
RT 5      15472
RT 110    14291
RT 405    11355
RT 605     9988
Name: road, dtype: int64

What was the least and most amount of crashes in a single day?

In [57]:
print(min(LA_data['same_day_crashes']))
print(max(LA_data['same_day_crashes']))

0
163


What was the weather like on those days?

In [58]:
LA_data[LA_data['same_day_crashes'] == 163]['rain_1h'].mean()

0.19472972972972977

In [59]:
LA_data[LA_data['same_day_crashes'] == 163]['visibility'].mean()

9759.256756756757

In [60]:
LA_data[LA_data['same_day_crashes'] == 0]['temp'].mean()

20.464966105068534

In [61]:
LA_data[LA_data['same_day_crashes'] == 163]['temp'].mean()

22.679459459459473

How many accidents involve something other than a car?

In [62]:
print(LA_data['pedestrian'].sum())
print(LA_data['bicycle'].sum())
print(LA_data['motorcycle'].sum())
print(LA_data['truck'].sum())

4977
3423
11016
26081


Although these numbers are only applicable to the highway accidents, it is quite surprising that the motorcycle accidents are about half the number of truck accidents, considering there are many more trucks on the road than motorcycles

### 9 Google Maps API<a id='9_Google_Maps_API'>

The google maps api will be used to get a list of possible routes for the user. The model will determine the safest route. To determine the model inputs and outputs, the google maps api response characteristics will be investigated.

In [63]:
gmaps.configure(api_key = maps_api_key)
DirMap = googlemaps.Client(key = maps_api_key)

In [64]:
LA_coords = (34.0522, -118.2437)
gmaps.figure(center = LA_coords, zoom_level = 10)

Figure(layout=FigureLayout(height='420px'))

To get directions from one location to another, the following code can be used,

In [65]:
Origin = "1111 S Figueroa St, Los Angeles, CA 90015"
Destination = "430 Normandie Pl, Los Angeles, CA 90004"

directions = DirMap.directions(Origin, Destination, mode = "driving", alternatives = True)

In [66]:
type(directions)

list

In [67]:
directions

[{'bounds': {'northeast': {'lat': 34.080595, 'lng': -118.2488444},
   'southwest': {'lat': 34.0426288, 'lng': -118.301229}},
  'copyrights': 'Map data ©2023 Google',
  'legs': [{'distance': {'text': '5.6 mi', 'value': 9056},
    'duration': {'text': '13 mins', 'value': 788},
    'end_address': '430 Normandie Pl, Los Angeles, CA 90004, USA',
    'end_location': {'lat': 34.078586, 'lng': -118.3012242},
    'start_address': '1111 S Figueroa St, Los Angeles, CA 90015, USA',
    'start_location': {'lat': 34.0426288, 'lng': -118.2662159},
    'steps': [{'distance': {'text': '492 ft', 'value': 150},
      'duration': {'text': '1 min', 'value': 21},
      'end_location': {'lat': 34.0436931, 'lng': -118.2652206},
      'html_instructions': 'Head <b>northeast</b> on <b>S Figueroa St</b> toward <b>W 11th St</b>',
      'polyline': {'points': 'm}wnEzzypUwAqAk@i@EEiAeA'},
      'start_location': {'lat': 34.0426288, 'lng': -118.2662159},
      'travel_mode': 'DRIVING'},
     {'distance': {'text': '0

The different routes are stored as dictionaries in a list. The steps can be found in directions[0]['legs'][0]['steps'] where n is the index of the route being investigated. The steps dictionary contains the start and end latitude and longitude as well as text stating the road being travelled on.

In [68]:
len(directions)

2

The API returned two routes. What are the roads beings travelled on in the first route?

In [69]:
roads = [0] * len(directions[0]['legs'][0]['steps'])
for i in range(len(directions[0]['legs'][0]['steps'])):
    roads[i] = directions[0]['legs'][0]['steps'][i]['html_instructions']
roads

['Head <b>northeast</b> on <b>S Figueroa St</b> toward <b>W 11th St</b>',
 'Turn <b>left</b> onto <b>Chick Hearn Ct</b>',
 'Turn <b>right</b> onto the <b>CA-110 N</b> ramp',
 'Keep <b>left</b> to continue toward <b>CA-110 N</b>',
 'Keep <b>left</b>, follow signs for <b>3rd St</b>/<wbr/><b>4th St</b>',
 'Keep <b>left</b> to continue toward <b>CA-110 N</b>',
 'Keep <b>left</b> and merge onto <b>CA-110 N</b>',
 'Take the <b>US-101 N</b>/<wbr/><b>US-101 S</b> exit toward <b>Ventura</b>/<wbr/><b>I-5 S</b>/<wbr/><b>I-10 E</b>/<wbr/><b>CA-60 E</b>',
 'Keep <b>left</b> at the fork, follow signs for <b>US-101 N</b>/<wbr/><b>Hollywood</b> and merge onto <b>US-101 N</b>',
 'Take exit <b>6A</b> for <b>Vermont Ave</b>',
 'Turn <b>left</b> onto <b>N Vermont Ave</b>',
 'Turn <b>right</b> onto <b>Rosewood Ave</b>',
 'Turn <b>left</b> onto <b>Normandie Pl</b><div style="font-size:0.9em">Destination will be on the left</div>']

The text above contains the roads that are being travelled on in this route. More will be done with this later, during feature engineering. But this is a good starting point for deciding what our explanatory features and target feature should look like.

### 10 OpenWeather API<a id='10_OpenWeather_API'>

The OpenWeather api will be used to get weather factors along the different routes supplied by the maps api. This information will also feed into the model, so it's characteristics must also be determined. It should be quite similar to the historic information that makes up the LA_weather dataframe. The possible file formats are json, html, and xml. Json will be used for the purpose of this project.

The current weather for a location in LA can be found as follows:

In [70]:
lat = LA_coords[0]
lon = LA_coords[1]
API_key = openWeather_api_key

LA_weather_response = requests.get(
    f'https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&exclude=minutely,hourly,monthly,alerts&appid={API_key}')

In [71]:
LA_weather_response.json()

{'lat': 34.0522,
 'lon': -118.2437,
 'timezone': 'America/Los_Angeles',
 'timezone_offset': -28800,
 'current': {'dt': 1676329156,
  'sunrise': 1676299202,
  'sunset': 1676338473,
  'temp': 289.23,
  'feels_like': 288.46,
  'pressure': 1017,
  'humidity': 60,
  'dew_point': 281.47,
  'uvi': 1.5,
  'clouds': 40,
  'visibility': 10000,
  'wind_speed': 5.14,
  'wind_deg': 190,
  'weather': [{'id': 802,
    'main': 'Clouds',
    'description': 'scattered clouds',
    'icon': '03d'}]},
 'daily': [{'dt': 1676318400,
   'sunrise': 1676299202,
   'sunset': 1676338473,
   'moonrise': 1676276700,
   'moonset': 1676314320,
   'moon_phase': 0.75,
   'temp': {'day': 287.93,
    'min': 283.3,
    'max': 289.23,
    'night': 285.61,
    'eve': 287.68,
    'morn': 283.32},
   'feels_like': {'day': 287.06,
    'night': 284.92,
    'eve': 286.86,
    'morn': 282.43},
   'pressure': 1015,
   'humidity': 61,
   'dew_point': 280.49,
   'wind_speed': 3.64,
   'wind_deg': 204,
   'wind_gust': 4.21,
   'weath

Knowing what these API resonses look like will make it much easier at the time of feature engineering to make informed decisions about how the data and model should be structured.

### 11 Save Data<a id='11_Save_Data'>

In [72]:
datapath = '../data'
save_file(LA_data, 'LA_data.csv', datapath)
save_file(LA_highway_collisions, 'LA_highway_collisions_cleaned.csv', datapath)
save_file(LA_nonhighway_collisions, 'LA_nonhighway_collisions_cleaned.csv', datapath)
save_file(LA_weather, 'LA_weather_cleaned.csv', datapath)

A file already exists with this name.

Do you want to overwrite? (Y/N)y
Writing file.  "../data/LA_data.csv"
A file already exists with this name.

Do you want to overwrite? (Y/N)y
Writing file.  "../data/LA_highway_collisions_cleaned.csv"
A file already exists with this name.

Do you want to overwrite? (Y/N)y
Writing file.  "../data/LA_nonhighway_collisions_cleaned.csv"
A file already exists with this name.

Do you want to overwrite? (Y/N)y
Writing file.  "../data/LA_weather_cleaned.csv"


### 12 Conclusion<a id='12_Conclusion'>

The data is now cleaned of all null values. The collision data, both highway and nonhighway, has been merged with the weather data, which will allow the model to take weather and time data into the risk calculations. 

The google maps api returns a number of routes when requested with a start and end location. The API can take addresses or a latitudes and longitudes for the endpoints, as well as returning street names and latitudes and longitudes. The street names/cooordinates are easily accessible through the returned list.

The weather api is almost the same as the bulk weather data from earlier in this notebook, differing in that the api response has a lot of unnecessary data. The useful data is luckily stored near the front (temp, humidity, visibility, etc...).